## Charts of Binding Energy 

The IAEA chart of nuclides datasets offer detailed nuclear properties like half-lives, decay modes, and energy levels, invaluable for research and applications in nuclear physics, medicine, and energy due to their comprehensive coverage and well validated datasets.

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))
df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all")
df["a"]=df["z"] + df["n"]
df["b_over_a"] = df["binding"]

import plotly.express as px

df["Atomic Number [A]"] = df["z"]  + df["a"]
df["Proton Number [Z]"] = df["z"]
df["Neutron Number [N]"] = df["n"]
df["Binding Energy [MeV]"] = df["binding"] / 1000.0
df["Symbol"] = df["symbol"] 
df["Dominant Decay"] = df["decay_1"] 


fig = px.scatter(df, x="Neutron Number [N]",y="Proton Number [Z]", color="Binding Energy [MeV]", hover_data=["Symbol","Neutron Number [N]","Proton Number [Z]","Binding Energy [MeV]", "Dominant Decay"])
fig.show()


def volume(A):
    return 15.56 * A

def surface(A):
    return -17.23 * A**(2.0/3.0)

def coulomb(A,Z):
    return -0.72*Z*(Z-1) / A**(1.0/3.0)

def assym(A,Z):
    p_aa = 93.14
    B_a=-p_aa*(Z-A/2)**2/A
    return B_a

def pair(A,Z):
    p_ap=12.0
    B_p=-p_ap/A**(1/2) 
    N=A-Z
    
    CV = np.array(Z%2 + N%2)
    B_p = np.array(B_p)
    B_p[ CV == 1 ] = -B_p[ CV == 1 ]
    B_p[ CV == 0 ] = 0

    return B_p

df["semf"] = (volume(df.a)+assym(df.a, df.z) + pair(df.a, df.z) + coulomb(df.a, Z=np.array(df.z)) + surface(df.a))/df.a/1E-3
df["LDM Ratio"] = df["semf"] / df["binding"]


df.loc[df["LDM Ratio"] < 0.975, "LDM Ratio"] = 0.975
df.loc[df["LDM Ratio"] > 1.025, "LDM Ratio"] = 1.025


fig = px.scatter(df, x="Neutron Number [N]",y="Proton Number [Z]", color="LDM Ratio", hover_data=["Symbol","Neutron Number [N]","Proton Number [Z]","Binding Energy [MeV]", "Dominant Decay"])
fig.show()


#df["Proton/Neutron Ratio"] = df["n"]/df["z"]
#fig = px.scatter(df, x="Atomic Number [A]",y="Binding Energy [MeV]", color="Proton/Neutron Ratio", hover_data=["Symbol","Neutron Number [N]","Proton Number [Z]","Binding Energy [MeV]", "Dominant Decay"])
#fig.show()



